In [23]:
# make algebra problems
# solve with Llama 8b
# check cot performance
# check no-cot performance
# tweak task prompt
# tweak algebra structure

import pandas as pd
import random
import transformers
import torch
import dotenv
import os

dotenv.load_dotenv()

True

In [24]:
def make_problem(digits, format=None):
    if format is None:
        n = random.randint(10**(digits-1), 10**digits-1)
        return str(n), n
    elif format[0] == '+':
        p1 = make_problem(digits, format[1][0])
        p2 = make_problem(digits, format[1][1])
        return f"({p1[0]}+{p2[0]})", p1[1] + p2[1]
    elif format[0] == '-':
        p1 = make_problem(digits, format[1][0])
        p2 = make_problem(digits, format[1][1])
        return f"({p1[0]}-{p2[0]})", p1[1] - p2[1]
    elif format[0] == '*':
        p1 = make_problem(digits, format[1][0])
        p2 = make_problem(digits, format[1][1])
        return f"({p1[0]}*{p2[0]})", p1[1] * p2[1]
    
def make_problem_set(digits, format, num_problems):
    return pd.DataFrame([make_problem(digits, format) for _ in range(num_problems)], columns=['problem', 'correct_solution'])

In [25]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=os.getenv('HF_TOKEN')
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def solve_problem(problem, sys_prompt="What is the value of this expression?"):
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": problem},
    ]

    outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=pipeline.tokenizer.eos_token_id
    )

    return outputs

def solve_problem_set(problem_set, sys_prompt="What is the value of this expression?"):
    solved = problem_set.copy()
    solved['solution'] = [solve_problem(row['problem'], sys_prompt) for _, row in problem_set.iterrows()]
    return [solve_problem(row['problem'], sys_prompt) for _, row in problem_set.iterrows()]

Loading checkpoint shards: 100%|██████████| 4/4 [00:37<00:00,  9.35s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
# def correct_rate(problem_set, sys_prompt):
#     processed = solve_problem_simple(problem_set['problem'], sys_prompt)
#     df_with_correct = df.copy()
#     df_with_correct['correct'] = df_with_correct.apply(correct, axis=1)

In [27]:
addition_2 = make_problem_set(2, ('+', (None, None)), 1)
addition_2_solutions = solve_problem_set(addition_2)